In [ ]:
import torch
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda, Compose
import torchvision.models as models
import numpy as np
from copy import deepcopy

import matplotlib.pyplot as plt
from random import randint
from torchvision.datasets import MNIST

import time

import os
os.chdir("/content/drive/MyDrive/Saravanan/")

In [ ]:
from datasets import return_dataset
[[mnist_train, mnist_test], [fashionmnist_train, fashionmnist_test], [cifar10_train, cifar10_test]] = return_dataset()
datasets = [[mnist_train, mnist_test], [fashionmnist_train, fashionmnist_test], [cifar10_train, cifar10_test]]

from initializations import initialize, negative_model, distorted_model
initlist = ['pre_trained', 'kaiming_uniform', 'kaiming_normal']



Files already downloaded and verified
Files already downloaded and verified


In [ ]:
#trainer function

import copy
from os import path, mkdir


def train_model(model, dataloaders, criterion, optimizer, num_epochs=25,
                folder = "/content/drive/MyDrive/Saravanan/training_data/", save_epochs=[]):
    since = time.time()

    val_acc_history = []
    train_acc_history = []
    
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:               
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                  # Get model outputs and calculate loss
                  # Special case for inception because in training it has an auxiliary output. In train
                  #   mode we calculate the loss by summing the final output and the auxiliary output
                  #   but in testing we only consider the final output.
                  outputs = model(inputs)
                  loss = criterion(outputs, labels)

                  _, preds = torch.max(outputs, 1)

                  # backward + optimize only if in training phase
                  if phase == 'train':
                      loss.backward()
                      optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)
            

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'val':
                val_acc_history.append(epoch_acc)
            elif phase == 'train':
                train_acc_history.append(epoch_acc)

        print()

        # saving requested epochs
        if epoch in save_epochs:        
          add = folder + "epoch_" + str(epoch) + '.pt'          
          torch.save(model, add)
          print("saved epoch", epoch, 'on', add[-60:])

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, val_acc_history, train_acc_history

def run_train(model, traindata, testdata, 
              lr=0.005, momemntum=0.9, epochsnum=20, batch_size=128, num_workers=2,
              rootadd = "/content/drive/MyDrive/Saravanan/training_data/",
              save_epochs=[]):

  name = model.name + '_init=' + model.initname + "_on_" + traindata.name + '_' + str(epochsnum) + "epochs"
  folderadd = rootadd + name
  
  
  # check folder does not exsit
  trynumber = 2
  testadd = folderadd
  while path.exists(testadd):
    testadd = folderadd + '_try' + str(trynumber)
    trynumber += 1
  folderadd = testadd + "/"
  mkdir(folderadd)

  initadd = folderadd + "initial_model.pt"
  torch.save(model, initadd)



  trainloader = torch.utils.data.DataLoader(traindata, batch_size=batch_size,
                                          shuffle=True, num_workers=num_workers)
  testloader = torch.utils.data.DataLoader(testdata, batch_size=batch_size,
                                         shuffle=False, num_workers=num_workers)
  dataloaders_dict = {'train': trainloader, 'val': testloader}
  
  criterion = torch.nn.CrossEntropyLoss()
  optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=momemntum)
  

  model.to(device)
  print('\n' * 3)
  print(name + '\n' + '='*len(name))
  model, test_history, train_history = train_model(model=model, dataloaders=dataloaders_dict,
                    criterion=criterion, optimizer=optimizer, num_epochs=epochsnum,
                    folder=folderadd, save_epochs=save_epochs)

  model.traindata = traindata.name
  model.train_history = train_history
  model.test_history = test_history
  model.epochs_trained = epochsnum  

  # save model
  modeladd = folderadd[:-1] + ".pt"
  torch.save(model, modeladd)
  return model


In [ ]:
from lenet import LeNet
# train models
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

initlist = ['kaiming_uniform', 'kaiming_normal']

epochsnum = 20
lr = 0.005
momemntum=0.9
batch_size=512
num_workers=2
save_epochs = [x for x in range(epochsnum) if x%5 == 0]

features_size = 512
classes_size = 10

for initname in initlist:

    model = LeNet(classes_size)
    model.name = 'lenet'
    # model.fc = torch.nn.Linear(features_size, classes_size, bias=True)
    initialize(model, initname)

    for data in datasets[1:2]:

      finalmodel = run_train(model=model, traindata=data[0], testdata=data[1],
                epochsnum=epochsnum, save_epochs=save_epochs)
      
      negativemodel = negative_model(finalmodel)
      run_train(model=negativemodel, traindata=data[0], testdata=data[1],
                epochsnum=epochsnum, save_epochs=save_epochs)
      
      distortedmodel = distorted_model(finalmodel)
      run_train(model=distortedmodel, traindata=data[0], testdata=data[1],
                epochsnum=epochsnum, save_epochs=save_epochs)
